## NBA Players/Coaches from 1976-2011

Data source :(https://www.kaggle.com/open-source-sports/mens-professional-basketball?select=basketball_awards_players.csv) 
Filtered data after the year 1976 since ABA, NBA merger took place in 1976 and the data before that is irrelavent/misleading.

created 3 data tables in the databse,

1.teams_master: contain each teams game winning/loosing summary of each year.Data files were merged to get coach name of each team on year.

2.team_summary:team level summary showing percentage of games that each team won

3.coaches mater: this table contains data of each coach, also included the awards they have received in year, if they received any. 

Data Analysis: We analised the length of years each team have been in NBA games:According to the data highest number of years were played by New York Knicks and Chicago Bulls, which is 45 years. when analysing this data it is evedent that some of the name changes of teams have a affect on the evaluation. To improve the accuracy of this evaluation team name changes should be incorporated in to the analysis.Also we analysed the % of games won by each team. 

In [76]:
import pandas as pd
import os
import sqlalchemy
import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect
from sqlalchemy import func
import getpass

In [54]:
# read all coaches details, filter after year 1976 daya
coaches_data=pd.read_csv("resources/basketball_coaches.csv")
coaches_inter=coaches_data[coaches_data['year']>=1976]
# drop unnecessary columns
coaches_df=coaches_inter[['coachID','year','tmID','won','lost']].copy()
coaches_df=coaches_inter.rename(columns={'year':'gameYear'})
coaches_df=coaches_df.rename(columns={'gameYear':'game_year', 'coachID':'coach_id','tmID':'team_id','lgID':'lg_id'})
coaches_df.dropna()
coaches_df

,coach_id,game_year,team_id,lg_id,stint,won,lost,post_wins,post_losses
526,attleal01,1976,GSW,NBA,1,46.0,36.0,5.0,5.0
527,badgeed01,1976,CHI,NBA,1,44.0,38.0,1.0,2.0
528,bayloel01,1976,NOJ,NBA,2,21.0,35.0,0.0,0.0
529,brownhe01,1976,DET,NBA,1,44.0,38.0,1.0,2.0
530,brownhu01,1976,ATL,NBA,1,31.0,51.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1639,popovgr01,2011,SAS,NBA,1,50.0,16.0,10.0,4.0
1640,caseydw01,2011,TOR,NBA,1,23.0,43.0,0.0,0.0
1641,corbity01,2011,UTA,NBA,1,36.0,30.0,0.0,4.0
1642,saundfl01,2011,WAS,NBA,1,2.0,15.0,0.0,0.0


In [55]:
# read awd coaches details, filter after year 1976 daya
awd_coaches_data=pd.read_csv("resources/basketball_awards_coaches.csv")
awd_coaches_inter=awd_coaches_data[awd_coaches_data['year']>=1976]
# drop unnecessary columns
awd_coaches=awd_coaches_inter[['coachID','year','award']].copy()
awd_coaches_final=awd_coaches.rename(columns={'year':'game_year', 'coachID':'coach_id'})
awd_coaches_final

,coach_id,game_year,award
25,nissato01,1976,NBA Coach of the Year
26,brownhu01,1977,NBA Coach of the Year
27,fitzsco01,1978,NBA Coach of the Year
28,fitchbi01,1979,NBA Coach of the Year
29,mckinja01,1980,NBA Coach of the Year
30,shuege01,1981,NBA Coach of the Year
31,nelsodo01,1982,NBA Coach of the Year
32,laydefr01,1983,NBA Coach of the Year
33,nelsodo01,1984,NBA Coach of the Year
34,fratemi01,1985,NBA Coach of the Year


In [56]:
# combine coach award details with coaches details and make one master database for coach details
coaches_master_data = pd.merge(coaches_df, awd_coaches_final, how="left", on=["coach_id", "game_year"])
# Remove duplictaes by coach ID, gameyear, tmID
#coaches_master_data.drop_duplicates(subset=["coach_id","game_year","tmeam_id"], inplace=True)
# create a copy of masterdata before setting the index
coaches_master_df=coaches_master_data.copy()
coaches_master_df.reset_index()
#coaches_master_df.index.name="in_id"
coaches_master_df

,coach_id,game_year,team_id,lg_id,stint,won,lost,post_wins,post_losses,award
0,attleal01,1976,GSW,NBA,1,46.0,36.0,5.0,5.0,NaN
1,badgeed01,1976,CHI,NBA,1,44.0,38.0,1.0,2.0,NaN
2,bayloel01,1976,NOJ,NBA,2,21.0,35.0,0.0,0.0,NaN
3,brownhe01,1976,DET,NBA,1,44.0,38.0,1.0,2.0,NaN
4,brownhu01,1976,ATL,NBA,1,31.0,51.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1113,popovgr01,2011,SAS,NBA,1,50.0,16.0,10.0,4.0,NBA Coach of the Year
1114,caseydw01,2011,TOR,NBA,1,23.0,43.0,0.0,0.0,NaN
1115,corbity01,2011,UTA,NBA,1,36.0,30.0,0.0,4.0,NaN
1116,saundfl01,2011,WAS,NBA,1,2.0,15.0,0.0,0.0,NaN


In [57]:
# read teams details
teams_data=pd.read_csv("resources/basketball_teams.csv")
teams_data

,year,lgID,tmID,franchID,confID,divID,rank,confRank,playoff,name,...,divWon,divLoss,pace,won,lost,games,min,arena,attendance,bbtmID
0,1946,NBA,BOS,BOS,NaN,ED,5,0,NaN,Boston Celtics,...,11,19,0,22,38,60,14500.0,Boston Garden,32767,BOS
1,1946,NBA,CHS,CHS,NaN,WD,1,0,F,Chicago Stags,...,17,8,0,39,22,61,14840.0,Chicago Stadium,0,CHS
2,1946,NBA,CLR,CLR,NaN,WD,3,0,R1,Cleveland Rebels,...,10,14,0,30,30,60,14600.0,Cleveland Arena,0,CLR
3,1946,NBA,DTF,DTF,NaN,WD,4,0,NaN,Detroit Falcons,...,8,16,0,20,40,60,14600.0,Detroit Olympia,0,DTF
4,1946,NBA,NYK,NYK,NaN,ED,3,0,SF,New York Knicks,...,13,17,0,33,27,60,14575.0,Madison Square Garden (III),32767,NYK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,1937,NBL,KNK,KNK,NaN,WE,6,0,NaN,Kankakee Gallagher Trojans,...,0,0,0,3,11,14,NaN,NaN,0,KNK
1532,1937,NBL,OSH,OSH,NaN,WE,1,0,LC,Oshkosh All-Stars,...,0,0,0,12,2,14,NaN,NaN,0,OSH
1533,1937,NBL,PGP,PGP,NaN,EA,3,0,NaN,Pittsburgh Pirates,...,0,0,0,8,5,13,NaN,NaN,0,PTP
1534,1937,NBL,WAR,CWH,NaN,EA,5,0,NaN,Warren Penns,...,0,0,0,3,9,12,NaN,NaN,0,WAR


In [58]:
#check all the column names
list(teams_data.columns.values)

['year',
 'lgID',
 'tmID',
 'franchID',
 'confID',
 'divID',
 'rank',
 'confRank',
 'playoff',
 'name',
 'o_fgm',
 'o_fga',
 'o_ftm',
 'o_fta',
 'o_3pm',
 'o_3pa',
 'o_oreb',
 'o_dreb',
 'o_reb',
 'o_asts',
 'o_pf',
 'o_stl',
 'o_to',
 'o_blk',
 'o_pts',
 'd_fgm',
 'd_fga',
 'd_ftm',
 'd_fta',
 'd_3pm',
 'd_3pa',
 'd_oreb',
 'd_dreb',
 'd_reb',
 'd_asts',
 'd_pf',
 'd_stl',
 'd_to',
 'd_blk',
 'd_pts',
 'o_tmRebound',
 'd_tmRebound',
 'homeWon',
 'homeLost',
 'awayWon',
 'awayLost',
 'neutWon',
 'neutLoss',
 'confWon',
 'confLoss',
 'divWon',
 'divLoss',
 'pace',
 'won',
 'lost',
 'games',
 'min',
 'arena',
 'attendance',
 'bbtmID']

In [65]:
#filter data after the year 1976 since ABA, NBA merger took place in 1976 and the data before that is irrelavent/misleading.
teams_data_inter=teams_data[teams_data['year']>=1976]
# drop unnecessary columns
teams=teams_data_inter[['year','tmID','divID','rank','name','divWon','divLoss','won','lost','games']].copy()
teams_final=teams.reset_index(drop=True)
teams_final=teams_final.rename(columns={'tmID':'team_id','year':'game_year','divID':'div_id','divWon':'div_won','divLoss':'div_loss','won':'games_won','lost':'games_lost','name':'team_name','games':'total_games','rank':'team_rank',})
teams_final

,game_year,team_id,div_id,team_rank,team_name,div_won,div_loss,games_won,games_lost,total_games
0,1976,ATL,CD,6,Atlanta Hawks,9,11,31,51,82
1,1976,BOS,AT,2,Boston Celtics,9,7,44,38,82
2,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82
3,1976,CHI,MW,2,Chicago Bulls,10,10,44,38,82
4,1976,CLE,CD,4,Cleveland Cavaliers,8,12,43,39,82
...,...,...,...,...,...,...,...,...,...,...
955,2011,SAC,PC,5,Sacramento Kings,0,0,22,44,0
956,2011,SAS,SW,1,San Antonio Spurs,0,0,50,16,0
957,2011,TOR,AT,4,Toronto Raptors,0,0,23,43,0
958,2011,UTA,NW,3,Utah Jazz,0,0,36,30,0


In [66]:
# pull in coach details to teams dataframe by merging with selected columns of coach details , with team details
# select only coach ID, year, team from coaches_master_data
slected_coach_data=coaches_master_data[['coach_id','game_year','team_id']].copy()

# merge the teams data with the selected coaches data so that teams data will contain the the coaches information.
teams_master_data_inter= pd.merge(teams_final, slected_coach_data, how="left", on=["team_id", "game_year"])
teams_master_data_inter

,game_year,team_id,div_id,team_rank,team_name,div_won,div_loss,games_won,games_lost,total_games,coach_id
0,1976,ATL,CD,6,Atlanta Hawks,9,11,31,51,82,brownhu01
1,1976,BOS,AT,2,Boston Celtics,9,7,44,38,82,heinsto01
2,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,locketa01
3,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,mackibo01
4,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,mullajo01
...,...,...,...,...,...,...,...,...,...,...,...
1113,2011,SAS,SW,1,San Antonio Spurs,0,0,50,16,0,popovgr01
1114,2011,TOR,AT,4,Toronto Raptors,0,0,23,43,0,caseydw01
1115,2011,UTA,NW,3,Utah Jazz,0,0,36,30,0,corbity01
1116,2011,WAS,SE,4,Washington Wizards,0,0,20,46,0,saundfl01


In [67]:
#drop one set of team ID resulted from the merge and also drop any null values
teams_master_data_inter.dropna()
teams_master_data_inter.replace('-', np.nan, inplace = True)
teams_master_data_inter = teams_master_data_inter.dropna()

In [69]:
#rename the columns
teams_master_data=teams_master_data_inter.rename(columns={'tmID':'team_id','gameYear':'game_year','divID':'div_id','divWon':'div_won','divLoss':'div_loss','won':'games_won','lost':'games_lost','name':'team_name','games':'total_games','rank':'team_rank','coachID':'coach_id'})
# make a copy for the database
teams_master_df=teams_master_data.copy()
teams_master_df.reset_index()
teams_master_df

,game_year,team_id,div_id,team_rank,team_name,div_won,div_loss,games_won,games_lost,total_games,coach_id
0,1976,ATL,CD,6,Atlanta Hawks,9,11,31,51,82,brownhu01
1,1976,BOS,AT,2,Boston Celtics,9,7,44,38,82,heinsto01
2,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,locketa01
3,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,mackibo01
4,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,mullajo01
...,...,...,...,...,...,...,...,...,...,...,...
1113,2011,SAS,SW,1,San Antonio Spurs,0,0,50,16,0,popovgr01
1114,2011,TOR,AT,4,Toronto Raptors,0,0,23,43,0,caseydw01
1115,2011,UTA,NW,3,Utah Jazz,0,0,36,30,0,corbity01
1116,2011,WAS,SE,4,Washington Wizards,0,0,20,46,0,saundfl01


In [56]:
# number of years each team has played
teams_master_data['team_name'].value_counts()

New York Knicks             45
Chicago Bulls               45
Denver Nuggets              44
New Jersey Nets             43
Los Angeles Lakers          43
Phoenix Suns                43
Cleveland Cavaliers         43
San Antonio Spurs           42
Golden State Warriors       41
Indiana Pacers              41
Philadelphia 76ers          41
Boston Celtics              40
Detroit Pistons             40
Milwaukee Bucks             39
Portland Trail Blazers      39
Seattle Supersonics         39
Atlanta Hawks               38
Houston Rockets             37
Dallas Mavericks            36
Utah Jazz                   36
Los Angeles Clippers        35
Sacramento Kings            34
Minnesota Timberwolves      28
Miami Heat                  26
Orlando Magic               26
Washington Bullets          25
Washington Wizards          19
Toronto Raptors             19
Charlotte Hornets           16
Memphis Grizzlies           16
Kansas City Kings           11
Charlotte Bobcats            9
New Orle

In [72]:
team_summary_totals=teams_master_data.groupby(teams_master_data['team_id'])['games_won','games_lost','total_games'].sum()
team_summary_totals

<ipython-input-72-b741040cda6d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  team_summary_totals=teams_master_data.groupby(teams_master_data['team_id'])['games_won','games_lost','total_games'].sum()


,games_won,games_lost,total_games
team_id,,,
ATL,1519,1549,2838
BOS,1782,1450,3002
BUF,117,211,328
CHA,152,324,246
CHH,587,661,1248
CHI,1765,1877,3412
CLE,1534,1944,3248
DAL,1436,1468,2674
DEN,1719,1841,3330


In [73]:
team_summary_totals['percentage_won']=team_summary_totals['games_won']/team_summary_totals['total_games']
team_summary_df=team_summary_totals.sort_values(by='percentage_won',ascending=False)
team_summary_df

,games_won,games_lost,total_games,percentage_won
team_id,,,,
OKC,198,196,164,1.207317
NOH,352,370,410,0.858537
LAL,2185,1229,3184,0.686244
SAS,2007,1389,3166,0.633923
UTA,1623,1281,2592,0.626157
CHA,152,324,246,0.617886
POR,1733,1401,2838,0.610641
PHO,1974,1504,3248,0.607759
BOS,1782,1450,3002,0.593604


In [29]:
password=getpass.getpass("enter password")
#username=input('enter username')
connection_string = f"postgres:{password}@localhost:5432/NBA_DB"
engine = create_engine(f'postgresql://{connection_string}')
collection=engine.connect()
inspector=inspect(engine)
inspector.get_table_names()

enter password········


['teams_summary', 'teams_master', 'coaches_master']

In [70]:
# load dataframe into database
teams_master_df.to_sql(name='teams_master',con=engine,if_exists="append", index=False)


In [51]:
coaches_master_df.to_sql(name='coaches_master',con=engine,if_exists="append", index=False)

In [74]:
team_summary_df.to_sql(name='teams_summary',con=engine,if_exists="append", index=True)

## Confirm data has been added by querying the tables

In [78]:
pd.read_sql_query('select * from teams_master', con=engine).head()

,game_year,team_id,div_id,team_rank,team_name,div_won,div_loss,games_won,games_lost,total_games,coach_id
0,1976,ATL,CD,6,Atlanta Hawks,9,11,31,51,82,brownhu01
1,1976,BOS,AT,2,Boston Celtics,9,7,44,38,82,heinsto01
2,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,locketa01
3,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,mackibo01
4,1976,BUF,AT,4,Buffalo Braves,6,10,30,52,82,mullajo01


In [80]:
pd.read_sql_query('select * from coaches_master', con=engine).head()

,coach_id,game_year,team_id,lg_id,stint,won,lost,post_wins,post_losses,award
0,attleal01,1976,GSW,NBA,1,46.0,36.0,5.0,5.0,None
1,badgeed01,1976,CHI,NBA,1,44.0,38.0,1.0,2.0,None
2,bayloel01,1976,NOJ,NBA,2,21.0,35.0,0.0,0.0,None
3,brownhe01,1976,DET,NBA,1,44.0,38.0,1.0,2.0,None
4,brownhu01,1976,ATL,NBA,1,31.0,51.0,0.0,0.0,None


In [81]:
pd.read_sql_query('select * from teams_summary', con=engine).head()

,team_id,games_won,games_lost,total_games,percentage_won
0,OKC,198,196,164,1.207317
1,NOH,352,370,410,0.858537
2,LAL,2185,1229,3184,0.686244
3,SAS,2007,1389,3166,0.633923
4,UTA,1623,1281,2592,0.626157
